# Objetivo

Extraer info relevante de archivos financieros, como son las llamadas trimestrales de ganancias de una empresa. Para ello, se plantean tres tareas principales:

1. `Extracción de entidades`: Se identifican entidades como personas, lugares y empresas, así como sus atributos, como roles o relaciones.
    1. Es una tarea de entity-extraction
2. `Identificación de enventos relevantes`: Se identifican eventos relevantes en el texto y se clasifican como positivos o negativos.
    1. Es una tarea que mezcla summarization con sentiment analysis.
3. `Búsqueda de información`: Se construye un pipeline para hacer preguntas al texto.
    1. Es una tarea de Question answering con RAG

No disponemos de un modelo especializado para este tipo de tareas ni de una gran db con la que pre-entrenar un modelo. Por ello, vamos a utilizar un LLM de gran tamaño, como GPT-3, como pto de partida.

En este caso, no se realizará fine-tuning del modelo, sino que se trabajará con el modelo base. Se experimentará con diferentes prompts y con el acercamiento RAG.



### `Sumario`

1. Preparar los datos
    * Cargar el doc
    * Dividir en párrafos
    * Contar nº de tokens por párrafo

2. Extraer entidades y relaciones
    * Preparar el prompt
    * Correr el prompt por el LLM
    * Formatear la salida
    * Comentarios

3. Identificar eventos relevantes
    * Preparar el prompt
    * Correr el prompt por el LLM
    * Formatear la salida
    * Comentarios

4. Obtener info mediante preguntas:
    * Modelo de embeddings
    * DB vectorial
    * Retrieval
    * Generation
    * Comentarios

In [2]:

from os import getenv, getcwd
from dotenv import load_dotenv
from langchain_openai import OpenAI


load_dotenv()

model = 'gpt-3.5-turbo-instruct'




max_tokens = 1000

llm = OpenAI(openai_api_key=getenv('OPEN_API_KEY', ''),
                 model= model,
                 temperature=0)

llm.max_tokens = max_tokens

In [3]:
from langchain.document_loaders import TextLoader
from os import getcwd
print(f"{getcwd()}\\assets\\growth_point_properties.md")

loader = TextLoader(f"{getcwd()}\\assets\\growth_point_properties.md")

docs = loader.load()

d:\Escritorio\Python Studio\Open Webinars\Langchain\proyecto final\assets\growth_point_properties.md


## 1.2 - Dividir en párrafos

En este caso voy a utilizar un método propio para dividir el texto en vez de utilizar un textsplitter de LangChain. La razón es que el texto parece estar mal formateado y un splitter normal no funcionaria como yo quiero.

Para identificar párrafos, `usamos una expresion linear que divide el texto cuando se encuentra un salto de línea \n seguido de una palabra mayúscula`.

Una mejor alternativa, sobretodo si vamos a tratar con múltiples documentos, sería preformattear el texto o buscar fuentes alternativas del mismo que si esten bien formateadas. ein embargo, vamos a considerar este ejercicio como una "Prueba de Concepto" y por ello, vamos a seguir un acercamiento más simple, trabajando directamente sobre el documento proporcionado.

In [4]:
from re import split as reSplit

def extract_paragraphs_from_txt(docs):
    paragraphs = []
    # Iteramos por cada pág del texto
    for page in docs:
        page_text = page.page_content

        #Usamos una expresión regular para identificar párrafos como aquellas lineas que empiezan por "\n" y luego una letra mayúscula
        page_paragraphs = reSplit(r'\n(?=[A-Z])', page_text)

        #Eliminamos párrafos vacíos
        page_paragraphs = [p.strip() for p in page_paragraphs if p.strip()]

        #Añadimos el párrafo a la lista
        paragraphs.extend(page_paragraphs)
    return paragraphs

#Dividimos el texto en párrafos
paragraphs = extract_paragraphs_from_txt(docs)

#Ahora "paragraphs" es una lista de strings, donde cada string es un párrafo
cleaned_paragraphs = [paragraph.replace('\n', '') for paragraph in paragraphs]

#Ahora añadimos un salto de página al inicio de cada párrafo para cuando utilicemos esta informacion en los prompts
cleaned_paragraphs = ['\n' + p for p in paragraphs]

cleaned_paragraphs

['\nGrowthpoint Properties Australia: Q4 2023 Earnings Call',
 '\nThank you for standing by and welcome to the Growthpoint Properties Australia FY 2023',
 '\nResults. All participants are in a listen-only mode. There will be a presentation followed by a\nquestion-and-answer session.',
 '\nI would now like to hand the conference over to Mr. Timothy Collyer, Managing Director. Please\ngo ahead.',
 '\nGood morning, and welcome to Growthpoint Properties Australia full year 2023 results. My\nname is Tim Collyer, Managing Director of Growthpoint. Joining me this morning are Michael',
 '\nGreen, Chief Investment Officer; and Dion Andrews, Chief Financial Officer. I will start this\nmorning with a brief overview of our results and strategy. Michael will then provide an update on\nour property portfolio followed by Dion, who will give a more detailed review of our financials.',
 "\nAnd finally, I'll provide a summary and outlook. We will then be happy to answer any questions\nyou may have.",
 "

## 1.3 - Contar el número de tokens por párrafo

Nuestro objetivo es pasar bloques de texto al LLM para que identifique entidades. Sin embargo, dentro del texto que vamos a pasar al LLM en forma de prompt va a haber información sobre como queremos que se extraigan. Es por ello que tenemos que medir bien los tamaño para que encajen dentro de la ventana de contexto.

Si ajustamos bien el texto, podemos reducir el número de llamadas que hacemos al modelo, y por tanto reducir la latencia de nuestro sistema.

Además saber cuantos tokens hay en cada párrafo nos permite saber cuanto cuesta (aproximadamente) cada llamada al modelo ya que en casos como el de GPT-3, el coste de las llamadas depende del número de tokens utilizados.

Podemos saber cual es el encoding de un modelo llamando al metodo `tiktoken.encoding_for_model()`:

In [5]:
from tiktoken import encoding_for_model
from pandas import DataFrame

def count_tokens (encoding, text):
    return len(encoding.encode(text))

encoding = encoding_for_model(model) #gpt-3.5-turbo-instruct

#Count tokens for each paragraph and store the results in a list of dictionaries
token_counts = []
for p in cleaned_paragraphs:
    num_tokens = count_tokens(encoding, p)
    token_counts.append({'Paragraph': p, 'Token Count': num_tokens})

#Create a DataFrame from the list of token counts
df = DataFrame(token_counts)
print(f'Nº total de tokens en el archivo: {df["Token Count"].sum()}')

df

Nº total de tokens en el archivo: 8301


,Paragraph,Token Count
0,\nGrowthpoint Properties Australia: Q4 2023 Ea...,15
1,\nThank you for standing by and welcome to the...,18
2,\nResults. All participants are in a listen-on...,26
3,\nI would now like to hand the conference over...,26
4,"\nGood morning, and welcome to Growthpoint Pro...",40
...,...,...
135,\nWoolworths as a major tenant across our port...,22
136,\nWoolworths approximately four to five years ...,50
137,\nGreat. Very interesting. Thanks for that.,10
138,\nThank you. That does conclude our question-a...,18


# 2 - Extraer entidades y relaciones

En esta tarea, combinamos las tareas NLP de "entity-extraction" y "relation-extraction" mediante un enfoque generativo.

* `Entity-extraction` (extracción de entidades). Se refiere a la identificación y clasificación de entidades mencionadas en el texto, como nombres de personas, lugares , organizaciones, etc.
* `Relation-extraction` (extracción de relaciones). Implica identificar y clasificar las relaciones semánticas entre entidades previamente extraidas. Por ejemplo, en una oración como "Bill gates es fundador de Microsoft", identificariamos la relación "fundador" entre las entidades "Bill Gates" y "Microsoft". Otras posibles relaciones que se pueden extraer indirectamente son: "Bill Gates es emprendedor", "Bill Gates es empresario", etc.


Para lograrlo, utilizaremos GPT-3 con Langchain, empleando una LLMChain. Como ya hemos visto, la LLMChain se compone de un PromptTemplate, un modelo y opcionalmente un formateador del output.

Nuestro objetivo implica identificar los siguientes elementos:

* Entidades, como `individuos y empresas`.
* Atributos pertinentes asociados con estas entidades, como `roles o relaciones`.


Nos centraremos principalmente en tres tipos de relaciones que existen entre entidades:

1. `<is_a>`: Esta relación es útil para definir la naturaleza de empresas, lugares o activos.
2. `<works_at>`: Esta relación indica el lugar de empleo de una persona.
3. `<has_position>`: Esta relación especifica el rol o posición que ocupa una persona dentro de la empresa con la que está asociada.


`Nota`: Si bien este enfoque podria ampliarse a otras relaciones más complejas, hemos optado por centrarnos en estas tres por claridad y con fines de demonstración.


In [6]:
entity_relationships_task = 'entity_relationships'

## 2.1 - Preparar el prompt

### Prompt base

Hemos dividio el prompt en 3 partes, cada una de ellas se encuentra en un archivo de texto:

* `base_template.txt`. Tiene el texto base que da contexto al modelo sobre lo que le vamos a pedir. Basicamente le indicamos que queremos hacer una tarea de extracción, donde las relaciones que encuentre el modelo en el texto deben ser indicadas en el formato ENTITY_1 <RELATIONSHIP> ENTITY_2.
* `pcpa.txt`. Indica el tipo de relaciones que nos interesan, y provee al modelo de explicaciones sobre cada una de ellas, para darle contexto.
* `pcpa_microsoft`. Contiene un ejemplo hecho a mano (one-shot learning) donde a partir de un texto, le indicamos que relaciones deberia extraer del mismo.

La idea de este prompt es explicar al LLM lo mejor posible la tarea para que pueda extraer correctamente entidades en el formato especificado. Como no hemos hecho ningún tipo de fine-tuning, hay que hacer algo de "prompt engineering".

In [7]:
from pathlib import Path

#Ruta al archivo base de plantilla para tareas de extracción de entidades
base_template_path = Path(f'{getcwd()}/prompts/templates/{entity_relationships_task}/base_template.txt')
relationships_template_path = Path(f'{getcwd()}/prompts/templates/{entity_relationships_task}/pcpa.txt')
example_template_path = Path(f'{getcwd()}/prompts/examples/{entity_relationships_task}/pcpa_microsoft.txt')

# Leer el contenido de cada plantilla desde el archivo
with open (base_template_path, 'r') as pf:
    base_template = pf.read()

with open (relationships_template_path, 'r') as pf:
    relationships_template = pf.read()

with open (example_template_path, 'r') as pf:
    example_prompt = pf.read()

#Combinar el texto del ej con el template del tipo de relaciones que nos interesan
prompt_text = relationships_template.format(example=example_prompt)

#Combinar el texto de la plantilla base ocn el prompt anterior
prompt_text = base_template.format(prompt=prompt_text)

# Contar el nº de tokens en el txt del prompt
n_prompt_tokens = count_tokens(encoding, prompt_text)
print(n_prompt_tokens)

208


In [8]:
print(prompt_text)

You are an expert in finance and financial news. You are great at extracting key business developments from earnings calls, news and financial documents.

When you are asked about the extraction of business developments from a text, you provide a list with the following format, where you rate them as positive and negative by giving them a score between -10 (very negative) and 10 (very positive):

<Business development summary> | <Score> | <Reason for score>

Here is what you should include in each list element:

* Business development summary. Write a headline explaining key aspects of the business development
* Score. Just write a number between -10 and 10 to indicate the positivity or negativity of the development
* Reason for score. Explanation about why the development is either positive or negative. Please include metrics and relevant information to support your reasoning.

Important: You should return your response that way. Do not write anything else outside of that. Do not incl

### Preparar chunks de texto a analizar
Con el texto anterior ya tenemos el contexto necesario para el LLM. Ahora lo que tenemos que hacer es preparar los chunks de texto de entrada para el modelo.

En este sentido, es muy importante adecuar de forma correcta el tamaño de los chunks, ya que si nos pasamos de tamaño el modelo nos dará un error (o no generará todo lo necesario). Hay 4 variables a considerar cuando decidimos el tamaño de los chunks:

El contexto máximo del modelo (en numero de tokens). En el caso de GPT-3 es 4096. Esto incluye tanto entrada como generación.
El número máximo de tokens que damos de margen al modelo para generar.
Además de estos dos, hay que tener que el prompt base ya tiene un numero de tokens (899) y que tiktoken no es deterministico (lo he observado empíricamente pero no he investigado la razón). Por ello el número máximo de tokens del chunk se puede estimar con la siguiente fórmula:

`max_tokens_per_chunk = MODEL_CONTEXT_LENGTH - MAX_GENERATION_LENGTH - EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY - n_prompt_tokens`

`max_tokens_per_chunk = 4097 - 1000 - 50 - 899`

`max_tokens_per_chunk = 2148`

Cómo máximo vamos a permitir chunks de 2147 tokens. Además, como nuestro objetivo es identificar entidades, vamos a evitar cortar párrafos por la mitad ya que podria afectar a las entidades o las relaciones.

In [47]:
MODEL_CONTEXT_LENGTH = 4097
MAX_GENERATION_LENGTH = 1000
EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY = 50 #not sure why, but I have run the same process multiple times and I have seen different tokenizations, need to double check this
llm.max_tokens = MAX_GENERATION_LENGTH

# Número máximo de tokens por chunk
max_tokens_per_chunk = MODEL_CONTEXT_LENGTH - MAX_GENERATION_LENGTH - EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY - n_prompt_tokens

print(f"Nº máx de tokens por chunk: {max_tokens_per_chunk}")

def generate_chunks(df, max_tokens_per_chunks):
    #Inicializar variables
    chunks = []
    current_chunk = []
    current_token_count = 0

    #Iterar a través de las filas del DF, que son párrafos del texto
    for idx, row in df.iterrows():
        paragraph = row['Paragraph']
        num_tokens = row['Token Count']

        # Si agregar el párrafo actual al fragmento actual no excede el límite
        if current_token_count + num_tokens <= max_tokens_per_chunks:
            current_chunk.append(paragraph)
            current_token_count += num_tokens
        else:
            #Agregar el párrafo actual a la lista de chunks
            if current_chunk:
                chunks.append("".join(current_chunk))
            #Comenzar un nuevo chunk con el párrafo actual
            current_chunk = [paragraph]
            current_token_count = num_tokens
    if current_chunk:
        chunks.append("".join(current_chunk))

    return chunks


Nº máx de tokens por chunk: 2839


In [48]:
chunks = generate_chunks(df, max_tokens_per_chunk)

In [49]:
total_token_count = 0

for i in range(0, len(chunks)):
    token_counts = count_tokens(encoding, chunks[i])
    print(f'{i}: {token_counts}')
    total_token_count += token_counts

print(f'Total: {total_token_count}')

0: 2771
1: 2732
2: 2686
Total: 8189


## 2.2 - Correr el prompt por el LLM

In [50]:
from time import time

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.callbacks import get_openai_callback

def llm_run(chain, query):
    start_time = time()

    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    end_time = time()
    execution_time = end_time - start_time
    print(f'Time taken: {round(execution_time, 2)} seconds')

    return result

#Crear la plantilla del prompt
prompt = PromptTemplate(
    input_variables=['input_text'],
    template=prompt_text + '\n{input_text}',
)

# Crear la cadena LLM simple
llm_chain = LLMChain(llm=llm, prompt=prompt)

er_outputs = []
for i in range(len(chunks)):
    print(f"Chunk {i}")
    output = llm_run(llm_chain, chunks[i])
    er_outputs.append(output)


Chunk 0


Spent a total of 3978 tokens
Time taken: 13.65 seconds
Chunk 1
Spent a total of 3939 tokens
Time taken: 13.5 seconds
Chunk 2
Spent a total of 3336 tokens
Time taken: 6.23 seconds


In [51]:
er_outputs

["\nOlympic Park, and a AUD 1.2 million lease break fee received in relation to 100 Skyring Terrace\nin Brisbane. Excluding these one-offs, NPI would have been down 1.3% on last year.\nThe group's interest cover ratio remains strong at 4.8 times, and our gearing remains at the low\nend of our target range at 29.3%. The group's weighted average debt maturity is 4.1 years, and\nwe have no debt maturing until December 2024. The group's weighted average cost of debt\nincreased to 2.9% from 2.6% last year, mainly due to the increase in market interest rates.\nMoving to slide 21, and the group's capital management. The group's balance sheet remains\nstrong, and we have a range of options available to us to fund our growth initiatives. We\ncompleted the AUD 100 million securities buyback program in May 2023, which was accretive to\nFFO per security. We have a number of unencumbered assets, which provide us with significant\nfunding flexibility. We have a strong relationship with our banks, an


Si bien el modelo nos devuelve tripletas de `"Entidad_1" <relacion> "Entidad_2"`, lo hace en forma de bloque de texto. Para poder utilizar dichas relaciones y almacenarlas por ejemplo en una base de datos en grafo como Node4J, tenemos que primero formatearlas correctamente...

In [52]:
print(er_outputs[0])


Olympic Park, and a AUD 1.2 million lease break fee received in relation to 100 Skyring Terrace
in Brisbane. Excluding these one-offs, NPI would have been down 1.3% on last year.
The group's interest cover ratio remains strong at 4.8 times, and our gearing remains at the low
end of our target range at 29.3%. The group's weighted average debt maturity is 4.1 years, and
we have no debt maturing until December 2024. The group's weighted average cost of debt
increased to 2.9% from 2.6% last year, mainly due to the increase in market interest rates.
Moving to slide 21, and the group's capital management. The group's balance sheet remains
strong, and we have a range of options available to us to fund our growth initiatives. We
completed the AUD 100 million securities buyback program in May 2023, which was accretive to
FFO per security. We have a number of unencumbered assets, which provide us with significant
funding flexibility. We have a strong relationship with our banks, and we have a r

## 2.3 - Formatear la salida

Dividimos aquellas lineas de texto que se encuentran en formato `"Entidad_1" <relacion> "Entidad_2"` y las almacenamos en un DataFrame de Pandas. Por supuesto, aquellas que no cumplen con las reglas son ignoradas.

In [53]:
from pandas import concat as pdconcat, Series as pdSeries
from re import findall as refindall, match as rematch


def split_relationship_text(text):
    #Dividir el texto basándose en "<" y ">" para separar la parte de la relación
    #match = rematch(r'(.+?)\s*\|\s*(\d+)\s*\|\s*(.+)', text)
    match = rematch(r'(.+?)\s*\|\s*(\d+)\s*\|\s*(.+)', text)

    print(match)
    

    if match:
        #Extraer entidades y relación
        entity1 = match.group(1).strip()
        relationship = match.group(2).strip()
        entity2 = match.group(3).strip()

        return entity1, relationship, entity2
    
    else:
        #Manejar de manera elegante la entrada no válida
        print(f'Formato no válido para: {text}')
        return None, None, None
    
er_outputs_dfs = []

#Crear DataFrames para cada salida en er_outputs
for out in er_outputs:
    er_outputs_dfs.append(DataFrame({"Triplet": out.split('\n')}))

#Concatenar los DF
er_output_df = pdconcat(er_outputs_dfs)

#Eliminar filas vacías y duplicados basados en la columna "Triplet"
er_output_df = er_output_df[~er_output_df["Triplet"].str.startswith("Resultado:") & (er_output_df["Triplet"] != "")]

#er_output_df = er_output_df[er_output_df["Triplet"] != ""]
er_output_df = er_output_df.drop_duplicates(["Triplet"]).reset_index(drop=True).copy()

#Aplicar la función al DF y crear nuevas columnas
er_output_df[['Entidad 1', 'Relación', 'Entidad 2']] = er_output_df['Triplet'].apply(
    lambda x: pdSeries(split_relationship_text(x))
)
    



None
Formato no válido para: Olympic Park, and a AUD 1.2 million lease break fee received in relation to 100 Skyring Terrace
None
Formato no válido para: in Brisbane. Excluding these one-offs, NPI would have been down 1.3% on last year.
None
Formato no válido para: The group's interest cover ratio remains strong at 4.8 times, and our gearing remains at the low
None
Formato no válido para: end of our target range at 29.3%. The group's weighted average debt maturity is 4.1 years, and
None
Formato no válido para: we have no debt maturing until December 2024. The group's weighted average cost of debt
None
Formato no válido para: increased to 2.9% from 2.6% last year, mainly due to the increase in market interest rates.
None
Formato no válido para: Moving to slide 21, and the group's capital management. The group's balance sheet remains
None
Formato no válido para: strong, and we have a range of options available to us to fund our growth initiatives. We
None
Formato no válido para: complete

In [54]:
er_output_df

,Triplet,Entidad 1,Relación,Entidad 2
0,"Olympic Park, and a AUD 1.2 million lease brea...",None,None,None
1,"in Brisbane. Excluding these one-offs, NPI wou...",None,None,None
2,The group's interest cover ratio remains stron...,None,None,None
3,end of our target range at 29.3%. The group's ...,None,None,None
4,we have no debt maturing until December 2024. ...,None,None,None
...,...,...,...,...
56,6. Bid/ask spread in office transactions | 4 |...,6. Bid/ask spread in office transactions,4,The bid/ask spread in office transactions is s...
57,7. Contribution from funds management business...,7. Contribution from funds management business,3,The contribution from the funds management arm...
58,8. Impact of higher interest rates on funds ma...,None,None,None
59,9. Low expiries in the next couple of years | ...,9. Low expiries in the next couple of years,7,The company has a low number of lease expiries...


# 3 - Identificar eventos relevantes

Esta tarea es una combinación de resumen y clasificación:

"Resumimos" el texto extrayendo eventos relevantes
Clasificamos los eventos como positivos o negativos.
Para ello, utilizaremos GPT-3 con Langchain, empleando una LLMChain. Como ya hemos visto, la LLMChain se compone de un PromptTemplate, un modelo y opcionalmente un formateador del output.

Nota: También podríamos probar un enfoque diferente donde dividimos el proceso en dos partes y utilizamos SimpleSequentialChain, donde la salida de un paso es la entrada al siguiente. Esto nos permitiría utilizar modelos diferentes para diferentes partes del proceso. Sería el enfoque que habría elegido si tuviera que trabajar con LLMs menos potentes (también más económicos), sobretodo si contase con más dato para entrenarlos.

La primera cadena generaría una lista de eventos.
La segunda cadena tomaría la lista de eventos como entrada y los clasificaría como positivos o negativos.

In [55]:
business_developments_task = 'business_developments'

## 3.1 - Preparar el prompt

### Prompt base

En este caso vamos a trabajar con un template más simple que en la tarea anterior ya que no vamos a proveer de ejemplos (zero-shot learning). En este caso simplemente proveemos de un texto explcativo para el modelo.

`Nota`: Realmente tengo preparado otro con ejemplo, pero haciendo pruebas con el me ha dado peores resultados, por lo que he decido utilizar zero-shot learning mejor que one-shot o few-shot learning

In [56]:
#Template base para la tarea de identificación de eventos relevantes
base_template_path = Path(f'{getcwd()}\\prompts\\templates\\{business_developments_task}\\base_template.txt')

with open(base_template_path, 'r') as prompt_file:
    base_template = prompt_file.read()

prompt_text = base_template

n_prompt_tokens = count_tokens(encoding, prompt_text)
print(n_prompt_tokens)

208


In [57]:
print(prompt_text)

You are an expert in finance and financial news. You are great at extracting key business developments from earnings calls, news and financial documents.

When you are asked about the extraction of business developments from a text, you provide a list with the following format, where you rate them as positive and negative by giving them a score between -10 (very negative) and 10 (very positive):

<Business development summary> | <Score> | <Reason for score>

Here is what you should include in each list element:

* Business development summary. Write a headline explaining key aspects of the business development
* Score. Just write a number between -10 and 10 to indicate the positivity or negativity of the development
* Reason for score. Explanation about why the development is either positive or negative. Please include metrics and relevant information to support your reasoning.

Important: You should return your response that way. Do not write anything else outside of that. Do not incl

# Preparar chunks de texto a analizar

Ya hemos definido previamente la función generate_chunks(). En este caso, solo tenemos que utilizarla correctamente:

max_tokens_per_chunk = MODEL_CONTEXT_LENGTH - MAX_GENERATION_LENGTH - EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY - n_prompt_tokens

max_tokens_per_chunk = 4097 - 1000 - 50 - 220

max_tokens_per_chunk = 2827


Nota: En este caso me encontré un error algo extraño de OpenAI (Error in OpenAICallbackHandler.on_llm_end callback: TypeError("unsupported operand type(s) for /: 'NoneType' and 'int'")), asi que modifique la longitud de los chunks y funciona...

In [60]:
max_tokens_per_chunk = 3201 - MAX_GENERATION_LENGTH - EXTRA_TOKENS_FOR_TOKENIZATION_VARIABILITY

chunks = generate_chunks(df, max_tokens_per_chunk)

total_token_count = 0

for i in range(0, len(chunks)):
    token_counts = count_tokens(encoding, chunks[i])
    print(f'{i}: {token_counts}')
    total_token_count += token_counts

print(f'"Total: {total_token_count}')

0: 2068
1: 2004
2: 2105
3: 2014
"Total: 8191


## 3.2 - Correr el prompt por el LLM

In [61]:
prompt = PromptTemplate(
    input_variables=['input_text'],
    template=prompt_text + "\n{input_text}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

bd_outputs = []
for i in range(len(chunks)):
    print(f"Chunks: {i}")
    output = llm_run(llm_chain, chunks[i])
    bd_outputs.append(output)

Chunks: 0


Spent a total of 3146 tokens
Time taken: 11.79 seconds
Chunks: 1
Spent a total of 2891 tokens
Time taken: 8.11 seconds
Chunks: 2
Spent a total of 3237 tokens
Time taken: 11.34 seconds
Chunks: 3
Spent a total of 2665 tokens
Time taken: 6.08 seconds


In [62]:
bd_outputs

[" We are proud of\nachieving a 92% satisfaction rating, which is a testament to our team's focus on tenant\nengagement.\n\nGrowthpoint Properties Australia: Q4 2023 Earnings Call\n\n1. Growthpoint's leasing performance was strong, with over 156,000 square meters leased equivalent to 11.2% of income, resulting in occupancy of 93% across the portfolio. | Score: 8 | This is a positive development as it shows a high occupancy rate and strong leasing performance, which will lead to sustainable income returns for securityholders.\n\n2. The group's goal remains to provide securityholders with sustainable income returns and capital appreciation over the long-term. | Score: 9 | This is a very positive development as it shows the company's commitment to its investors and its long-term strategy for growth and value creation.\n\n3. In financial year 2023, Growthpoint purchased the GSO building in Dandenong with a 9.4-year WALE and divested 333 Ann Street, Brisbane with a 3.7-year WALE. | Score: 7

# 3.3 - Formatear la salida

Dividimos aquellas lineas de texto que se encuentran en formato "Evento" | -10 a 10 | "Razón del score" y las almacenamos en un DataFrame de Pandas. Por supuesto, aquellas que no cumplen con las reglas son ignoradas.

In [63]:
def split_business_development_text(text):
    parts = text.split('|')
    if len(parts) == 3:
        business_development_summary = parts[0].strip()
        score = parts[1].strip()
        reason_for_score = parts[2].strip()
        return business_development_summary, score, reason_for_score
    else:
        print(f'Invalid format for: {text}')
        return None, None, None
    
bd_output_dfs = []
for output in bd_outputs:
    bd_output_dfs.append(DataFrame({'RAW Output': output.split('\n')}))

bd_output_df = pdconcat(bd_output_dfs)
bd_output_df = bd_output_df[bd_output_df["RAW Output"] != '']
bd_output_df = bd_output_df.drop_duplicates(["RAW Output"]).reset_index(drop=True).copy()

#Creamos nuevas columnas
bd_output_df[['Business development', 'Score', 'Explanation']] = bd_output_df['RAW Output'].apply(
    lambda x: pdSeries(split_business_development_text(x))
)
bd_output_df = bd_output_df.dropna()

Invalid format for:  We are proud of
Invalid format for: achieving a 92% satisfaction rating, which is a testament to our team's focus on tenant
Invalid format for: engagement.
Invalid format for: Growthpoint Properties Australia: Q4 2023 Earnings Call
Invalid format for:  So we're not seeing
Invalid format for: any real change in the tenant profile that we're attracting to our assets.
Invalid format for: Okay. Got you. And just last one for me, just on the industrial portfolio. So I understand you're
Invalid format for: looking to sell two assets. But just curious, how do you think about the rest of the portfolio
Invalid format for: given the strong demand for industrial assets right now? And do you think there's any
Invalid format for: opportunity to sell more assets in the near term?
Invalid format for: Thanks, Solomon. It's Tim here. Look, we're always looking at our portfolio and we're always
Invalid format for: looking at opportunities to recycle capital. We've got a very strong 

In [64]:
bd_output_df

,RAW Output,Business development,Score,Explanation
4,1. Growthpoint's leasing performance was stron...,1. Growthpoint's leasing performance was stron...,Score: 8,This is a positive development as it shows a h...
5,2. The group's goal remains to provide securit...,2. The group's goal remains to provide securit...,Score: 9,This is a very positive development as it show...
6,"3. In financial year 2023, Growthpoint purchas...","3. In financial year 2023, Growthpoint purchas...",Score: 7,This is a positive development as it shows the...
7,4. Capital management remains a sharp focus gi...,4. Capital management remains a sharp focus gi...,Score: 8,This is a positive development as it shows the...
8,5. The group's property portfolio value decrea...,5. The group's property portfolio value decrea...,Score: -6,This is a negative development as it shows a d...
9,"6. Despite challenging market conditions, dist...","6. Despite challenging market conditions, dist...",Score: 7,This is a positive development as it shows the...
10,7. The positive momentum in the industrial mar...,7. The positive momentum in the industrial mar...,Score: 9,This is a very positive development as it show...
11,8. The acquisition of Fortius Funds Management...,8. The acquisition of Fortius Funds Management...,Score: 9,This is a very positive development as it show...
12,9. The group's office portfolio is reduced by ...,9. The group's office portfolio is reduced by ...,Score: -5,This is a negative development as it shows a d...
13,10. Tenant engagement is a major focus for Gro...,10. Tenant engagement is a major focus for Gro...,Score: 10,This is a very positive development as it show...


## 3.4 - Comentarios

### Variabilidad en el formato de los salidas

Es evidente que, incluso aun estableciendo la temperatura del modelo a 0, se generan respuestas en diversos formatos. Por ejemplo:

* Algunas respuestas comienza con listas numeradas
* Las explicaciones de los scores a veces se presentan directamente y otras empizan como "This is a positive development..."

Para mitigar esta variabilidad y lograr un formato más consistente, tenemos varios opciones:

* Mejorar los prompts
* Hacer fine-tuning del modelo

En mi opinión, lo que daría mejores resultados es hacer un adecuado fine-tuning del modelo con ejemplos preparados en el formato deseado.

Alternativamente, podríamos considerar volver a ejecutar secciones de texto que no se han formateado según lo previsto. Podríamos usar una configuración de temperatura más alta durante estas ejecuciones y verificar si la salida se alinea con nuestras expectativas.

Salidas Innecesarias (Desviación de las reglas establecidas en el prompt): Como se evidencia, el modelo ocasionalmente genera texto que no sigue las instrucciones que proporcionamos. Afortunadamente, ignoramos estas salidas innecesarias durante el proceso de formateo. Para abordar este problema, deberíamos investigar por qué ocurre esto y explorar mejoras potenciales para evitar tales desviaciones de nuestras pautas en el futuro.



# 4 - Obtener info mediante preguntas

Esta tarea tiene como objetivo poner en práctica lo que hemos visto en la parte de Retrieval Augmented Generation (RAG). Especificamente, es una tarea de pregunta-respuesta con contexto, donde el contexto se almacena en una base de datos vectorial.

Como elementos relevantes de la misma, utilizaremos:
* BBDD: Chroma (en memoria RAM)
* Modelo de embeddings:
* Modelo de lenguaje

En este caso vamos a preparar una aplicación muy simple, por lo que no vamos a introducir memoria, multi-query u otras de las mejoras que hemos discutido en la seccion de RAG avanzado.

## 4.1 - Modelo de embeddings

In [65]:
from langchain_openai import OpenAIEmbeddings
from os import getenv


openai_embeddings_model = OpenAIEmbeddings(
    model = "text-embedding-3-large",
    openai_api_key = getenv('OPEN_API_KEY', '')
)

## 4.2 - BBDD vectorial

In [67]:
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma

chunks = df["Paragraph"].tolist()

documents = []
for c in chunks:
    doc = Document(page_content=c, metadata={'source': 'local'})
    documents.append(doc)

db = Chroma.from_documents(documents, openai_embeddings_model)

## 4.3 - Retrieval

Ahora escribamos la lógica real de la aplicación. Queremos crear una aplicación sencilla que

1. Tome una pregunta del usuario
2. Busque documentos relevantes para esa pregunta
3. Pase los documentos recuperados y la pregunta inicial a un LLM
4. Devuelva una respuesta

Primero, necesitamos definir nuestra lógica para buscar en los documentos. LangChain define una interfaz Retriever que envuelve un índice que puede devolver Documentos relevantes dado una uery. Su funcionamiento es similar al de db.similarity_search().

El tipo más común de Retriever es el VectorStoreRetriever, que utiliza las capacidades de búsqueda de similitud de un vector store para facilitar la recuperación.

In [68]:
#Creamos el retriever, que devuelve los k textos más similares a la pregunta
retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': 3})

In [69]:
#Corremos una pregunta de ejemplo
retrieved_docs = retriever.invoke("Who are the Chief executives in Growth Point Properties?")

In [71]:
#Documentos recuperados
retrieved_docs

[Document(page_content='\nGood morning, and welcome to Growthpoint Properties Australia full year 2023 results. My\nname is Tim Collyer, Managing Director of Growthpoint. Joining me this morning are Michael', metadata={'source': 'local'}),
 Document(page_content='\nThe Growthpoint AUD 4.8 billion balance sheet portfolio consists of the modern office and\nindustrial assets, which are principally leased to government listed and large private tenants and\nprovide the bedrock of a resilient cash flow for our securityholders.', metadata={'source': 'local'}),
 Document(page_content='\nGreen, Chief Investment Officer; and Dion Andrews, Chief Financial Officer. I will start this\nmorning with a brief overview of our results and strategy. Michael will then provide an update on\nour property portfolio followed by Dion, who will give a more detailed review of our financials.', metadata={'source': 'local'})]

Parece que recupera correctamente la información, la cual se encuentra dividida en 3 párrafos (si bien en el texto original realmente es uno) por los problemas del PDF que hemos comentado anteriormente (no se cargan correctamente los saltos de linea)

## 4.4 - Generation

Una vez tenemos los documentos que otorgarán de contexto al LLM, debemos pasarle dicha información dentro de un prompt (junto a la pregunta).

In [72]:
model = 'gpt-3.5-turbo-instruct'




max_tokens = 1000

llm = OpenAI(openai_api_key=getenv('OPEN_API_KEY', ''),
                 model= model,
                 temperature=0)

llm.max_tokens = max_tokens

In [73]:
retrieved_docs

[Document(page_content='\nGood morning, and welcome to Growthpoint Properties Australia full year 2023 results. My\nname is Tim Collyer, Managing Director of Growthpoint. Joining me this morning are Michael', metadata={'source': 'local'}),
 Document(page_content='\nThe Growthpoint AUD 4.8 billion balance sheet portfolio consists of the modern office and\nindustrial assets, which are principally leased to government listed and large private tenants and\nprovide the bedrock of a resilient cash flow for our securityholders.', metadata={'source': 'local'}),
 Document(page_content='\nGreen, Chief Investment Officer; and Dion Andrews, Chief Financial Officer. I will start this\nmorning with a brief overview of our results and strategy. Michael will then provide an update on\nour property portfolio followed by Dion, who will give a more detailed review of our financials.', metadata={'source': 'local'})]

In [74]:
# Plantilla específica para Question-Answering
custom_rag_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.

{context}

Question: {question}

Answer:"""

custom_rag_prompt = PromptTemplate(
    input_variables=['context', 'question'],
    template=custom_rag_template
)

#Crear la cadena LLM simple
qa_chain = LLMChain(llm=llm, prompt = custom_rag_prompt)

In [75]:
def qa_run(chain, query):
    start_time = time()

    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    end_time = time()
    execution_time = end_time - start_time
    print(f'Time taken: {round(execution_time, 2)} seconds')
    return result

context = ''
for i in range (0, len(retrieved_docs)):
    doc = retrieved_docs[i]
    context += f"\nDocument #{i}:"
    context += doc.page_content

#La pregunta
question = "Who are the Chief executives in Growth Point Properties?"

# La query resultante que mandamos al modelo
query = {
    'context': context,
    'question': question
}

qa_run(qa_chain, query)

Spent a total of 246 tokens
Time taken: 0.86 seconds


' Tim Collyer, Michael Green, and Dion Andrews are the Chief executives in Growth Point Properties.'

## 4.5 - Comentarios

Podemos incrementar la complejidad de la aplicación añadiendo funcionalidades tales como:

* Streaming. En vez de devolver la respuesta de golpe una vez ha sido generada, podemos ir devolviendola token a token según se genera (similar al comportamiento de Chat-GPT).
* Memoria. Este sistema está pensado para interacciones puntuales sobre el modelo. No permite una conversación continuada. Para ello, tendriamos que implementar un historial del chat (i.e., una memoria de corto plazo).
* Fuentes. Si bien para este ejemplo particular no tiene mucho sentido ya que solo hay un documento sobre el cual estamos haciendo preguntas. Podria darse el caso de que queremos extender la funcionalidad de la aplicacion para analizar un documento de una compañia, sino múltiples documentos, en tal caso, estaria bien que el modelo nos indicase cual es el documento (y que parrafos del mismo) ha utilizado para generar su respuesta.

A su vez, si los párrafos fueran muy largos, deberiamos hacer estimaciones del número de tokens para poder ajustar lo mejor posible las queries (similar a las secciones 2 y 3 de este proyecto)